<a href="https://colab.research.google.com/github/ErikaAZapanaZ/matematica-computacional/blob/main/Examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CELDA 1: CREAR LIBRERIA.PY
%%writefile /content/libreria.py
import numpy as np

def interCambiaFilas(A,fil_i,fil_j):
    A[[fil_i,fil_j],:]=A[[fil_j,fil_i],:]

def operacionFila(A,fil_m, fil_piv, factor):
    A[fil_m,:]= A[fil_m,:]- factor*A[fil_piv,:]

def reescalaFila(A,fil_m,factor):
     A[fil_m,:]= factor*A[fil_m,:]

def escalonarMatriz(A):
 nfila=A.shape[0]
 ncol=A.shape[1]

 for j in range(0,nfila):
     for i in range(j+1, nfila):
         ratio=A[i,j]/A[j,j]
         operacionFila(A,i,j,ratio)

def escalonaConPiv(A):
    nfila=A.shape[0]
    ncol=A.shape[1]

    for j in range(0,nfila):
      imax=np.argmax(np.abs(A[j:nfila,j]))
      interCambiaFilas(A,j+imax,j)

      for i in range(j+1, nfila):
         ratio=A[i,j]/A[j,j]
         operacionFila(A,i,j,ratio)

def ceros_en_diagonal(A):
    nfila=A.shape[0]
    ncol=A.shape[1]
    for i in range(nfila):
     for j in range(ncol):
      if i == j:
        A[i][j] = 0
    return A

def sustRegresiva(A,b):
   N= b.shape[0]
   x= np.zeros((N,1))
   for i in range (N-1,-1,-1):
      x[i,0]=(b[i,0]-np.dot(A[i,i+1:N],x[i+1:N,0]))/A[i,i]
   return x

def sustProgresiva(A,b):
   N= b.shape[0]
   x= np.zeros((N,1))
   for i in range (0,N):
      x[i,0]=(b[i,0]-np.dot(A[i,0:i],x[0:i,0]))/A[i,i]
   return x

def GaussElimSimple(A,b):
   Ab=np.append(A,b,axis=1)
   escalonarMatriz(Ab)
   A1=Ab[:,0:Ab.shape[1]-1].copy()
   b1=Ab[:,Ab.shape[1]-1].copy()
   b1=b1.reshape(b.shape[0],1)
   x=sustRegresiva(A1,b1)
   return x

def GaussElimWithPiv(A,b):
   Ab= np.append(A,b,axis=1)
   escalonaConPiv(Ab)
   A1= Ab[:,0:Ab.shape[1]-1].copy()
   b1= Ab[:,Ab.shape[1]-1].copy()
   b1= b1.reshape(b.shape[0],1)
   x= sustRegresiva(A1,b1)
   return x

def LUdeComposition(A):
   nrows= A.shape[0]
   U=A.copy()
   L= np.eye(nrows,nrows,dtype=np.float64)
   for col in range (0,nrows-1):
      for row in range (col+1,nrows):
         mult=U[row,col]/U[col,col]
         L[row,col]=mult
         operacionFila(U,row,col,mult)
   return (L,U)

def solveByLU(A,b):
   LU= LUdeComposition(A)
   L=LU[0]
   U=LU[1]
   Y=sustProgresiva(L,b)
   X=sustRegresiva(U,Y)
   return X

def transpuesta(A):
    A = A.astype(float)
    filas, columnas = A.shape
    At = np.zeros((columnas, filas))
    for i in range(filas):
        for j in range(columnas):
            At[j, i] = A[i, j]
    return At



def escalonaConPivot(A):
    nfila, ncol = A.shape
    nCambios = 0
    tol = 1e-12

    for j in range(min(nfila, ncol)):
        imax_rel = np.argmax(np.abs(A[j:nfila, j]))
        imax = j + imax_rel

        if abs(A[imax, j]) < tol:
            continue

        if imax != j:
            interCambiaFilas(A, j, imax)
            nCambios += 1

        for i in range(j + 1, nfila):
            if abs(A[j, j]) > tol:
                ratio = A[i, j] / A[j, j]
                operacionFila(A, i, j, ratio)

    return nCambios

def Rank(A):
   A= A.copy().astype(float)
   escalonaConPivot(A)
   tolerancia= 1e-12
   rango= np.sum(np.abs(np.diag(A))>tolerancia)
   return rango

def multiDeterminante(A):
   d=1
   for k in range(len(A)):
      d*=A[k][k]
      if nCambios % 2 == 1:
        d = -d
   return d

def Determinante(A):
   A = A.astype(np.float64)
   n = A.shape[0]
   escalonaConPiv(A)
   return multiDeterminante(A)

def Inversa(A):
    n = A.shape[0]
    M = np.hstack((A.astype(float), np.eye(n)))
    escalonaConPiv(M)

    for j in range(n - 1, -1, -1):
        piv = M[j, j]
        if abs(piv) < 1e-12:
            print(" La matriz no tiene inversa.")
            return None
        M[j, :] /= piv
        for i in range(j):
            ratio = M[i, j]
            M[i, :] -= ratio * M[j, :]

    A_inv = M[:, n:]
    return A_inv

def qr_factorization(A):
    A = A.astype(float)
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        R[j, j] = np.sqrt(np.sum(A[:, j]**2))

        if R[j, j] == 0:
            print("A has linearly dependent columns")
            return None, None

        Q[:, j] = A[:, j] / R[j, j]

        for k in range(j + 1, n):
            R[j, k] = np.dot(Q[:, j], A[:, k])
            A[:, k] = A[:, k] - Q[:, j] * R[j, k]

    return Q, R

def solveByQR(A, b):
    QR = qr_factorization(A)
    Q=QR[0]
    R=QR[1]
    Qtb = np.dot(transpuesta(Q), b)
    x = sustRegresiva(R, Qtb)
    return x

Overwriting /content/libreria.py


In [ ]:
import numpy as np
import libreria as lb
import time

#A=np.array([[1,2],[3,4]])
#print(A)

#print("\n", A[[2,4],:]) # los : indica toda la fila  extrae toda la fila 2 y 4

n=500 #tamaño del sistema lineal
#A1=np.random.uniform(0,5, size=(n, n)) # matriz de orden 4x4 con numeros flotantes de 0 a 5
A=np.random.rand(3,3)
b= np.random.uniform(0,6, size=(3,1))
#A[i][i]=0. # aqui en la parte de ratio la division se da entre cero y las respuestas salen nan o inf
#print(A)
#operacionFila(A,2,0,2)
#interCambiaFilas(A,1,2)
#reescalaFila(A,0,3)
#escalonarMatriz(A)
#B=ceros_en_diagonal(A)
#print("\n",B)
#escalonaConPiv(B)
#print("\n",B)
"""
A=np.array([[1,2,1],[1,0,1],[0,1,2]])
b=np.array([[0],[2],[1]])
print(A)
print(b)"""
start_time=time.perf_counter() #para ver cuanto tiempo se toma en hacer los calculos
sol= lb.GaussElimSimple(A,b)
end_time= time.perf_counter()
elapsed_time=end_time-start_time
print(f'Tiempo transcurrido: {elapsed_time:.4f} segundos')
#print("\n",sol)
residuo= A@sol - b
#print("\n",residuo)
print("norma del residuo",np.linalg.norm(residuo))

x= np.array([[2,1,-1,2],[4,5,-3,6],[-2,5,-2,6],[4,11,-4,8]])
y= np.array([[1,-2,-1,3],[-1,3,-2,-2],[2,0,1,1],[1,-2,2,3]])
z= np.array([[1,4,0,0],[2,3,0,1],[0,4,1,5],[0,0,2,3]])
m= np.array([[0,4,-2,4],[-6,2,10,0],[5,8,-5,2],[0,-2,1,0]])
ra= np.array([[1,2,3,-1],[0,5,1,7],[2,-1,5,-9],[4,-2,10,-18]])
re= np.zeros((n,n))
descpmqr= np.array([[1,1,2],[1,0,-2],[-1,2,3]])
descomp2= np.array([[2,1,0,0],[1,2,1,0],[0,1,2,1],[0,0,1,2]])
invers=np.array([[2,-1,3],[4,0,1],[6,1,2]])
d=lb.Determinante(A)
print("determinante", d)
d2=np.linalg.det(A)
print("determinatecon numpy2", d2)

r=lb.Rank(A)
print("el rango es:",r)
r1= np.linalg.matrix_rank(A)
print("el rango con numpy es:",r1)

r2=lb.Inversa(invers)
print("la inversa es:",r2)
r3= np.linalg.inv(invers)
print("la inversa con numpy es:",r3)

QR= lb.qr_factorization(descomp2)
Q=QR[0]
print("Q: \n", Q)
R=QR[1]
print("R: \n",R)
print("A=QR: \n", Q@R)

QRE= lb.qr_factorization(A)
Q=QRE[0]
R=QRE[1]
print("A=QR: \n", Q@R)

QR2=np.linalg.qr(descomp2)
Q2=QR2[0]
print("Q: \n", Q2)
R2=QR2[1]
print("R: \n",R2)
print("A=QR: \n", Q2@R2)
#print("la matriz Q2 es:\n",Q2)


n1=4
A1= np.random.rand(n1,n1)
b1= np.random.rand(n1,1)
X= lb.solveByQR(A1,b1)
print(X)

A2=np.array([[1,0,0,1],[0,1,-1,-1],[0,-1,1,0],[1,-1,0,0]])
b2=np.array([[0],[0],[1],[0]])
X1= lb.solveByQR(A2,b2)
print(X1)
A3=np.array([[4,-1,0,0,0],[-1,4,-1,0,0],[0,-1,4,-1,0],[0,0,-1,4,-1],[0,0,0,-1,4]])
b3=np.array([[100],[200],[200],[200],[100]])
X2= lb.solveByQR(A3,b3)
print(X2)

print("||Ax-b||_1:\n", np.linalg.norm(A2@X1-b2,1))



Tiempo transcurrido: 0.0002 segundos
norma del residuo 4.839668336948288e-15
determinante -0.0678695507440197
determinatecon numpy2 0.06786955074401961
el rango es: 3
el rango con numpy es: 3
la inversa es: [[-0.08333333  0.41666667 -0.08333333]
 [-0.16666667 -1.16666667  0.83333333]
 [ 0.33333333 -0.66666667  0.33333333]]
la inversa con numpy es: [[-0.08333333  0.41666667 -0.08333333]
 [-0.16666667 -1.16666667  0.83333333]
 [ 0.33333333 -0.66666667  0.33333333]]
Q: 
 [[ 0.89442719 -0.35856858  0.19518001 -0.18257419]
 [ 0.4472136   0.71713717 -0.39036003  0.36514837]
 [ 0.          0.5976143   0.58554004 -0.54772256]
 [ 0.          0.          0.68313005  0.73029674]]
R: 
 [[2.23606798 1.78885438 0.4472136  0.        ]
 [0.         1.67332005 1.91236577 0.5976143 ]
 [0.         0.         1.46385011 1.95180015]
 [0.         0.         0.         0.91287093]]
A=QR: 
 [[ 2.00000000e+00  1.00000000e+00 -3.76090281e-17  3.28179204e-17]
 [ 1.00000000e+00  2.00000000e+00  1.00000000e+00 -4.